In [ ]:
"""
MODIFICADO DE cpv2010arg.py en https://github.com/abenassi/pyredatam/blob/1480c481feb0698d54b59c3c17e52661a8c793df/pyredatam/cpv2010arg.py
"""
import os
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from pyvirtualdisplay import Display
import pandas as pd
from bs4 import BeautifulSoup
import requests
from collections import OrderedDict
import json
import time


BASE_URL = "https://censos2017.inei.gob.pe/bininei/RpWebStats.exe/CmdSet?BASE=CPV2017DI&ITEM=PROGRED&lang=esp"
DRIVER_PATH = "C:\\Users\\Dieguinchi\\Desktop\\chromedriver.exe"

def make_query(query, url=BASE_URL): # hace consulta "query" a redatam a través de procesador estadístico online
    options = webdriver.ChromeOptions()#carga configuración del webdriver
    driver = webdriver.Chrome(options=options, executable_path=DRIVER_PATH)# el driver debe estar en archivos del pc. puede descargarse de https://chromedriver.chromium.org/downloads
    driver.get(url) # abre pagina web de redatam
    query_input = driver.find_element_by_tag_name("textarea")# ubica linea de comandos
#     query_input.send_keys(query.decode("utf-8", "ignore"))
    query_input.send_keys(query) # escribe consulta en línea de comandos
    submit = driver.find_element_by_name("Submit") #Busca botón "Ejecutar"
    submit.click()# clickea en "Ejecutar" y ejecuta consulta
    WebDriverWait(driver, 999999).until(lambda driver: len(driver.find_elements_by_xpath("//*[contains(text(),'Descargar en formato Excel')]")) == int(len(var_in)))#espera 999 segundos o a que se muestren todas las tablas solicitadas, es decir, debe mostrar Descargar en formato Excel la misma cantidad de veces que la cantidad de variables de la solicitud 
    html = driver.find_element_by_id("tab-output")# obtiene unicamente la tabla de resultados
    html = html.get_attribute('outerHTML')# obtiene el html de la tabla de resultados
    driver.close() # cierra navegador
    return html # devuelve html con tabla de resultados

def make_dataframe(html): # crea dataframe con resultados
    temp_table = pd.read_html(html) # lee todos los dataframes de la tabla de resultados
    return temp_table # devuelve lista con dataframes

merged = pd.concat(make_dataframe(make_query(query))) # concatena dataframes de la tabla de resultados en un solo dataframe
Excelwriter = pd.ExcelWriter("tablas_redatam2017.xlsx",engine="xlsxwriter") #exporta en excel dataframe final
merged.to_excel(Excelwriter)
Excelwriter.save()